In [ ]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "$NOTEBOOK_TYPE$"
PYTHON_VERSION = "$PYTHON_VERSION$"
IPYTHON_VERSION = "$IPYTHON_VERSION$"

In [ ]:
import warnings
from pathlib import Path

import requests


warnings.filterwarnings('error', module='davos')

if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [ ]:
import re
from contextlib import redirect_stdout
from io import StringIO

import davos
import IPython
import pkg_resources
from davos.core.core import run_shell_command
from davos.core.exceptions import DavosConfigError

from utils import mark, raises, run_tests

In [ ]:
IPYTHON_SHELL = get_ipython()

# tests for `davos.core.config`

In [ ]:
def test_config_is_singleton():
    assert davos.core.config.DavosConfig() is davos.config

In [ ]:
def test_deactivate_reactivate_configlevel():
    assert davos.config.active
    
    davos.config.active = False
    assert not davos.config.active
    
    with raises(NameError, match=re.escape("name 'smuggle' is not defined")):
        smuggle ast
        
    davos.config.active = True
    assert davos.config.active

In [ ]:
@mark.colab
def test_auto_rerun_disabled_colab():
    with raises(DavosConfigError):
        davos.config.auto_rerun = True

In [ ]:
@mark.jupyter
def test_confirm_install_disabled_noninteractive():
    match = re.escape(
        "'davos.config.confirm_install': field may not be 'True' in "
        "noninteractive mode"
    )
    try:
        davos.config.noninteractive = True
        with raises(DavosConfigError, match=match):
            davos.config.confirm_install = True
    finally:
        # ensure noninteractive mode & prompting are disabled after test
        davos.config.noninteractive = False
        davos.config.confirm_install = False

In [ ]:
def test_environment():
    msg = f"davos.config.environment is '{davos.config.environment}'"
    if NOTEBOOK_TYPE == 'colab':
        assert davos.config.environment == 'Colaboratory', msg
    elif IPython.version_info[0] < 7:
        assert davos.config.environment == 'IPython<7.0', msg
    else:
        assert davos.config.environment == 'IPython>=7.0', msg

In [ ]:
def test_ipython_shell_is_global_obj():
    assert davos.config.ipython_shell is IPYTHON_SHELL

In [ ]:
def test_ipython_shell_is_read_only():
    match = re.escape("'davos.config.ipython_shell': field is read-only")
    with raises(DavosConfigError, match=match):
        davos.config.ipython_shell = None

In [ ]:
@mark.colab
def test_ipython_shell_is_colab_shell():
    google_version = pkg_resources.get_distribution('google').version
    assert isinstance(IPYTHON_SHELL, google.colab._shell.Shell), (
        "Qualified name for Colab interactive shell class has changed (google "
        "module may have been recently updated).\n\tShell type:\t"
        f"{type(IPYTHON_SHELL)}\n\tgoogle version:\t{google_version}"
    )

In [ ]:
@mark.jupyter
def test_noninteractive_warns_if_confirm_install():
    match = re.escape(
        "noninteractive mode enabled, setting `confirm_install = False`"
    )
    try:
        davos.config.confirm_install = True
        with raises(UserWarning, match=match):
            davos.config.noninteractive = True
    finally:
        # ensure prompting & noninteractive mode are disabled after test
        davos.config.noninteractive = False
        davos.config.confirm_install = False

In [ ]:
@mark.colab
def test_noninteractive_mode_disabled_colab():
    match = re.escape(
        "'davos.config.noninteractive': noninteractive mode not available "
        "in Colaboratory"
    )
    with raises(DavosConfigError, match=match):
        davos.config.noninteractive = True

In [ ]:
def test_pip_executable_exists():
    pip_exe_path = Path(davos.config.pip_executable)
    assert pip_exe_path.is_file(), f"{pip_exe_path} does not exist"

In [ ]:
@mark.jupyter
def test_pip_executable_in_kernel_env():
    pip_exe = davos.config.pip_executable
    assert pip_exe.endswith('/envs/kernel-env/bin/pip'), (
        "expected davos.config.pip_executable path to end in "
        f"'/envs/kernel-env/bin/pip'. Path is '{pip_exe}'"
    )

In [ ]:
def test_set_pip_executable_fails_no_exist():
    bad_path = '/fake/path/to/bin/pip'
    match = re.escape(
        "'davos.config.pip_executable': No such file or directory: "
        "'/fake/path/to/bin/pip'"
    )
    with raises(DavosConfigError, match=match):
        davos.config.pip_executable = bad_path

In [ ]:
def test_set_pip_executable_fails_is_dir():
    tmpdir = Path.cwd().joinpath('tmpdir')
    tmpdir.mkdir(parents=False, exist_ok=False)
    match = re.escape(
        f"'davos.config.pip_executable': '{tmpdir}' is not an executable file"
    )
    try:
        with raises(DavosConfigError, match=match):
            davos.config.pip_executable = tmpdir
    finally:
        tmpdir.rmdir()

In [ ]:
def test_smuggled_dict_initially_empty():
    assert davos.config.smuggled == {}, (
        "expected davos.config.smuggled to be empty. Found: "
        f"'{davos.config.smuggled}'"
    )

In [ ]:
def test_smuggled_is_read_only():
    match = re.escape("'davos.config.smuggled': field is read-only")
    with raises(DavosConfigError, match=match):
        davos.config.smuggled = {'foo': 'bar'}

In [ ]:
def test_suppress_stdout():
    try:
        davos.config.suppress_stdout = True
        with redirect_stdout(StringIO()) as tmp_stdout:
            run_shell_command('echo "hello world"', live_stdout=None)
            tmp_stdout = tmp_stdout.getvalue()
        assert tmp_stdout == '', (
            f"expected stdout to be suppressed. Found: '{tmp_stdout}'"
        )
    finally:
        davos.config.suppress_stdout = False

In [ ]:
run_tests()